In [ ]:
#simple bert classifier

In [ ]:
# !pip install transformers
# !pip install torch
# !pip install pytorch-lightning

In [7]:
 !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 4.9/4.9 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 3.9/3.9 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 GB 1.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.1
    Uninstalling torch-2.1.1:
      Successfully uninstalled torch-2.1.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\zacha\\anaconda3\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [1]:
import torch

# If there's a GPU available...
torch.cuda.is_available()    


True

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")